This notebook script will print out the per channel accuracy of a multi-channel architecture model

In [30]:
import numpy as np

import pandas as pd

In [26]:
path = '/gpfs/gpfs_gl4_16mb/b9p111/fdl_sw/experiments_results/throwaway_40/0050_throwaway_40_test_predictions.npy'

results = np.load(path)

wavelengths = [
  "0094",
  "0131",
  "0171",
  "0193",
  "0211",
  "0304",
  "0335"]
num_channels = len(wavelengths)
tolerance = 0.05

In [42]:
primary_metrics = [0] * num_channels
for c in range(num_channels):
  gt = results[:, c, 0]
  pred = results[:, c, 1]
  
  diff = np.abs(pred - gt)
  
  batch_size = diff.shape[0]
  n_elems = batch_size
  
  primary_metrics[c] = np.sum(diff < tolerance, dtype=np.float32) / n_elems

df = pd.DataFrame(np.array(primary_metrics)[np.newaxis, :], columns=wavelengths)

In [43]:
df

,0094,0131,0171,0193,0211,0304,0335
0,0.872648,0.914616,0.968162,0.858177,0.767004,0.916064,0.771346
